In [32]:
import os
import glob
import rasterio
import rioxarray as rx
import numpy as np
import geopandas as gpd
from multiprocessing import Pool, cpu_count
from shapely.ops import unary_union

In [8]:
os.chdir('../..')

In [13]:
os.chdir('DADOS\TIF\CORRECAO PROFLORA - ANOS PRIORITARIOS')

In [14]:
os.getcwd()

'c:\\Users\\rogerio.siqueira\\Documents\\DEMANDAS_ROGERIO\\APM\\DADOS\\TIF\\CORRECAO PROFLORA - ANOS PRIORITARIOS'

In [17]:
with rasterio.open("Reclass_Mapbiomas2021-APM_PROFLORA.tif") as src:
    # bnd = rasterio.band(src, 1)
    print(src.tags())

{'AREA_OR_POINT': 'Area'}


In [18]:
src.close()

In [33]:
def union(x):
    return unary_union(x)

with rx.open_rasterio("Reclass_Mapbiomas2021-APM_PROFLORA.tif") as apm:
    x, y, dn = apm.x.values, apm.y.values, apm.values
    x, y = np.meshgrid(x, y)
    x, y, dn = x.flatten(), y.flatten(), dn.flatten()

    gdf_apm = gpd.GeoDataFrame.from_dict(
        {
            'DN':dn,
            'x':x,
            'y':y
        }
    )
    gdf_apm
    
    geom_arr = []
 
    apm_vector = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_xy(gdf_apm['x'], gdf_apm['y'], crs=apm.rio.crs))

    # Converting GeoSeries to list of geometries
    geoms = list(apm_vector)
 
    # Converting geometries list to nested list of geometries
    for i in range(0, len(geoms), 10000):
        geom_arr.append(geoms[i:i+10000])
 
    # Creating multiprocessing pool to perform union operation of chunks of geometries
    with Pool(cpu_count()) as p:
        geom_union = p.map(union, geom_arr)
 
    # Perform union operation on returned unioned geometries
    total_union = unary_union(geom_union)
 
    # Creating GeoDataFrame for total_union
    union_vector_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(total_union))

In [ ]:
union_vector_gdf

In [30]:
apm_vector

,geometry
0,POINT (151290.000 8284800.000)
1,POINT (151320.000 8284800.000)
2,POINT (151350.000 8284800.000)
3,POINT (151380.000 8284800.000)
4,POINT (151410.000 8284800.000)
...,...
5773307,POINT (235710.000 8223390.000)
5773308,POINT (235740.000 8223390.000)
5773309,POINT (235770.000 8223390.000)
5773310,POINT (235800.000 8223390.000)
